##### Importar Dependencias

In [ ]:
from os import listdir
from scr.util import *

##### Definir el documento de entrada del inicio y fin de los lanzamientos y los nombres de las carpetas de salida

In [ ]:
data_doc_path = "./data/SCUvsIND.txt"

default_videos_output_path = "./output/Clips/"
dein_videos_output_path = "./output/Clips_dein/"
detected_images_output_path = "./output/BallDetected_yolo/"
not_detected_images_output_path = "./output/BallNotDetected_yolo/"

default_name = "SCUvsIND_02"
video = "./data/video/SCUvsIND_02.mts"

model_path = "../YOLOv8/yolov8m.pt"

# MASTER
MAX_VIDEOS_PROCS = 10000000


#### Recortar los videos

In [ ]:
count = 1


for M, N in load_doc_data(data_doc_path):
    trim_video(  # Margen de Error M - 5  -->  N + 5
        video,
        default_videos_output_path
        + default_name
        + format_name_default_output_video(count),
        M - 5,
        N + 5,
        1,
    )
    count += 1
    if count > MAX_VIDEOS_PROCS:
        break

print("Trim Videos Done")

#### Desentrelazar los clips

In [ ]:
for clip in listdir(default_videos_output_path):
    print(format_name_dein_output_video(clip))
    output_name = dein_videos_output_path + format_name_dein_output_video(clip)
    deinterlace_video(default_videos_output_path + clip, output_name)
    

#### Cargar el modelo y dependencias

In [ ]:
import cv2
import torch
from ultralytics import YOLO

model = YOLO(model_path)
print(torch.cuda.is_available())

In [ ]:
clips = listdir(dein_videos_output_path)
clips.sort()

for clip in clips:
    count = 1
    cap = cv2.VideoCapture(dein_videos_output_path + clip)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        result = model(frame)

        output_img = not_detected_images_output_path

        if 32 in result[0].boxes.cls:
            output_img = detected_images_output_path

        tmp = format_name_output_img(clip, count)
        
        print(tmp)

        cv2.imwrite(output_img + tmp, frame)
        count += 1